In [4]:
import numpy as np
from pulp import *
import scipy
l = [[2,4,5],[5,5,8]]
print(np.array(l)[1,1])


5


In [19]:
prices = [np.random.uniform(20, 50) for _ in range(24)]
prices.extend([35]*24)

prices = tuple(prices)
print(prices)

(48.79759036480179, 25.235157762555467, 30.612072410524355, 34.42972688896395, 31.247234559495013, 33.544926630234656, 32.09457874149777, 41.223969635920206, 26.030504538366525, 32.82409315709194, 31.270635380678705, 40.31553617565526, 32.31724416713819, 25.0205571075428, 28.97406205247487, 42.571176746335794, 47.95094812251286, 43.2251668201364, 21.83414759859731, 33.47075880101432, 30.08043985951921, 46.60004598597456, 33.6416448986934, 33.74793912234351, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35)


In [17]:

# Define the problem
prob = LpProblem("MaximizeSum", LpMaximize)

# number of hours
NH: int = 48

# max amount stored per hour
Smax = 50
# max amount produced per hour
Emax = 50

# reservoir capacity
Rmax = 500
# reservoir initial reserves
R0 = 250 

#target fractional reserve
f = 0.5

# gen efficiency
eff_g = 0.9

# pump efficiency
eff_st = 0.8

# transmission efficiency 
h = 0.95


# Define the decision variables - Amount of electricity produced at this hour 
E = LpVariable.dicts("E", range(NH), lowBound=0, upBound=Emax, cat='Float')
S = LpVariable.dicts("S", range(NH), lowBound=0, upBound=Smax, cat='Float')

# Define the objective function
prob += lpSum([prices[i] *(E[i]*h - S[i]*1/h)for i in range(NH)])

# Define the constraints

for k in range(NH):
    prob += (R0 + lpSum([S[i]*eff_st-E[i]*1/eff_g for i in range(k)])) <= Rmax
    prob += (R0 + lpSum([S[i]*eff_st- E[i]*1/eff_g for i in range(k)])) >= 0

prob += lpSum([E[i]*1/eff_g -S[i]*eff_st for i in range(NH)]) == int(R0 -f*Rmax)

# Solve the problem
status = prob.solve()

# Print the solution
print("Status:", LpStatus[status])
print("Objective value:", value(prob.objective))
for i in range(NH):
    print(f'price {prices[i]}')
    print(f"E[{i}] = {value(E[i] - S[i])}")



Status: Optimal
Objective value: 2872.4793116499814
price 23.99183888788814
E[0] = -50.0
price 27.248954489417102
E[1] = -50.0
price 27.33672231299645
E[2] = -16.6667
price 37.26693340806041
E[3] = 0.0
price 34.84229049255588
E[4] = 0.0
price 49.17954390509617
E[5] = 50.0
price 22.190833578937482
E[6] = -50.0
price 34.12058896010965
E[7] = 0.0
price 46.33915571417947
E[8] = 50.0
price 48.98791471981862
E[9] = 50.0
price 38.80138692819932
E[10] = 0.0
price 26.294425489007534
E[11] = -50.0
price 42.53138678380065
E[12] = 50.0
price 34.81035066199344
E[13] = 0.0
price 36.978800910201855
E[14] = 0.0
price 47.78708661899566
E[15] = 50.0
price 23.864549981189384
E[16] = -50.0
price 27.57618333158377
E[17] = 0.0
price 45.92173313124086
E[18] = 50.0
price 22.908881550110692
E[19] = -50.0
price 41.73614334135914
E[20] = 0.0
price 20.693189770133102
E[21] = -50.0
price 22.491966174487665
E[22] = -50.0
price 36.133994901446
E[23] = 0.0
price 35
E[24] = 0.0
price 35
E[25] = 0.0
price 35
E[26] = 0.

In [ ]:
from scipy.optimize import linprog

